In [ ]:
# Example 20: gain profile of GPS antenna @ 1575 Mz +- 80 MHz
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import asyncio
from rtlsdr import RtlSdr
import matplotlib.pyplot as plt
import numpy as np
import keyboard
import time
import json


# -------------- global variables -------------------        

L1 = 1575.42e6         
FSTEP = 0.4e6
FN = 200
FMIN = L1 - FN*FSTEP
FMAX = L1 + FN*FSTEP

# configuration SDR
BIAS_TEE = True         # set to True with GPS antenna
SAMPLE_RATE = 2.048e6   
FREQ_CORRECTION = -1    # in ppm
GAIN = 50               # 0..49.6; tuner GAIN in dB

MSEC = 16               # time in ms between two plots 
NSMP = MSEC*2048        # number of samples per cycle    

SAVE_RESULT = False     # set True for saving results

FILE_SAVE = '../data/gpsAntenna'
FILE_LOAD = '../data/240228_gpsAntenna.json'


# ------- Exception handling  -------------------

import linecache
import sys

EXC = []

def printException():
    global EXC
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    EXC.append(('EXCEPTION IN ({}, LINE {} "{}"): {}'\
                .format(filename, lineno, line.strip(), exc_obj)))        
    
# ------- save results ----------------

def saveData(data,filename):
    with open(filename,'w') as file:
        json.dump(data,file,indent=2)

def loadData(filename):
    with open(filename,'r') as file:
        data = json.load(file)
    return data


# ------- process & plot data ---------


def plot(x,y):
    x = np.asarray(x)
    line1.set_xdata(x/1e6)
    line1.set_ydata(y) 
    fig.canvas.draw_idle()   
    

# ------- plot data -------------------


async def processData():   
    sdr = RtlSdr()
    sdr.set_bias_tee(BIAS_TEE)
    sdr.sample_rate = SAMPLE_RATE
    sdr.freq_correction = FREQ_CORRECTION    
    sdr.center_freq = FMIN   
    sdr.gain = GAIN     
    try:
        xf,yf = [],[]
        data = []
        i = 0
        F = FMIN
        while i <= 2*FN +1:
            sdr.center_freq = F
            y = sdr.read_samples(NSMP)
            y = np.abs(y)**2
            rms = np.sqrt(np.mean(y))
            xf.append(F)
            yf.append(rms)
            data.append((F,rms))
            plot(xf,yf)
            F += FSTEP
            i += 1            
            await asyncio.sleep(0.01)
        sdr.stop()
        sdr.close()
    except:
        printException()
    finally:
        if SAVE_RESULT:
            saveData(data,FILE_SAVE)


# --------------- prepare plot ------------------        
                    
fig,(ax,bx) = plt.subplots(1,2,figsize=(10,4))
fig.canvas.header_visible = False

# plot of measurement
line1, = ax.plot([0],[0],'o',ms=1)
ax.grid()
ax.set_ylim(0,0.25)
ax.axline((L1/1e6,0),(L1/1e6,0.2),c='red',lw=0.5,ls='--')
ax.set_xlim(FMIN/1e6,FMAX/1e6)
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Amplitude (a.u.)')
ax.set_title('GPS Antenna: Gain Profile')

# for comparison; result from author
data = loadData(FILE_LOAD)
x,y = zip(*data)
x = np.asarray(x)
bx.plot(x/1e6,y,'o',ms=1)
bx.set_ylim(0,0.25)
bx.axline((L1/1e6,0),(L1/1e6,0.2),c='red',lw=0.5,ls='--')
bx.grid()
bx.set_xlabel('Frequency (MHz)')
bx.set_ylabel('Amplitude (a.u.)')
bx.set_title('Comparison: Measurement 28.02.24')

plt.tight_layout()

print('Press q to exit!')

plt.show()

# ------ Main ---------------------------

# start async tasks
loop = asyncio.get_event_loop()
task2 = loop.create_task(processData())
